In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-calihan/time_series_covid19_confirmed_global.csv
/kaggle/input/covid19-calihan/covid_19_clean_complete.csv
/kaggle/input/covid19-calihan/time_series_covid19_deaths_global.csv
/kaggle/input/covid19-calihan/time_series_covid19_recovered_global.csv


In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
import csv 
import operator 
import datetime as dt  
from pandas import DataFrame as df
%matplotlib inline

import math
import os
import json

plt.rcParams['font.family'] = 'NanumGothic'

In [3]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, TimestampedGeoJson

In [4]:
covid = pd.read_csv('/kaggle/input/covid19-calihan/covid_19_clean_complete.csv', engine='python')
confirmed_df = pd.read_csv('/kaggle/input/covid19-calihan/time_series_covid19_confirmed_global.csv', encoding='cp949')
deaths_df = pd.read_csv('/kaggle/input/covid19-calihan/time_series_covid19_deaths_global.csv', encoding='cp949')
recovered_df = pd.read_csv('/kaggle/input/covid19-calihan/time_series_covid19_recovered_global.csv', encoding='cp949')

# 최근 국가별 총 현황을 보여주는 데이터셋
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [5]:
# covid
confirmed_df
# deaths_df
# recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,584,609,634,663,678,712,726,736,750,766
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,717,717,723,723,731,738,738,743,743,743
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,24,24,25,25,25,25,26,27,27,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,...,4,4,4,5,5,5,6,6,34,34
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,8,8


In [6]:
# 대소문자 구분이 어려워. 매핑
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
deaths_df.columns = map(str.lower, deaths_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

In [7]:
# 이름 긴것도 좀 정리해봐
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
deaths_df = deaths_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [8]:
confirmed_df

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,584,609,634,663,678,712,726,736,750,766
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,717,717,723,723,731,738,738,743,743,743
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,24,24,25,25,25,25,26,27,27,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,...,4,4,4,5,5,5,6,6,34,34
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,8,8


In [9]:
# 전체 확진자를 정리해보자
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [10]:
# HTML로 보여주기
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:15px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:15px;margin-left:20px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:15px; margin-left:20px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

## 국가별 확산 시각화

In [11]:
# 상위 n 개의 국가를 보여주는 레이아웃

fig = go.FigureWidget( layout=go.Layout() )


def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # 이거 안해주면 그래프 안나오는데 왜그런지 모르겠음. 인터넷이 시킴
widgets.VBox([fig], layout=ipywLayout)

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-05-06 08:32:29,40.000000,-100.000000,1204475,71078,189791,972538,365.583692,NaN,NaN,5.901160,840,USA
161,Spain,2020-05-06 08:32:29,40.463667,-3.749220,219329,25613,123486,70230,469.104947,NaN,NaN,11.677890,724,ESP
10,Italy,2020-05-06 08:32:29,41.871900,12.567400,213013,29315,85231,98467,352.309890,NaN,NaN,13.762071,380,ITA
16,United Kingdom,2020-05-06 08:32:29,55.000000,-3.000000,196243,29501,926,165816,289.077260,NaN,NaN,15.032893,826,GBR
6,France,2020-05-06 08:32:29,46.227600,2.213700,170694,25538,52859,92297,261.505770,NaN,NaN,14.961276,250,FRA
7,Germany,2020-05-06 08:32:29,51.165700,10.451500,167007,6993,137400,22614,199.330552,NaN,NaN,4.187250,276,DEU
13,Russia,2020-05-06 08:32:29,61.524000,105.318800,165929,1537,21327,143065,113.701041,NaN,NaN,0.926300,643,RUS
174,Turkey,2020-05-06 08:32:29,38.963700,35.243300,129491,3520,73285,52686,153.536202,NaN,NaN,2.718336,792,TUR
39,Brazil,2020-05-06 08:32:29,-14.235000,-51.925300,115953,7958,48221,59774,54.550867,NaN,NaN,6.863126,76,BRA
90,Iran,2020-05-06 08:32:29,32.427908,53.688046,99970,6340,80475,13155,119.021890,NaN,NaN,6.341903,364,IRN


    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [12]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [13]:
# 상위 n 개에 해당하는 산포도
def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [14]:
# 총 확진자와 총 사망자를 비교해주는 그래프

def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, deaths_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();
    
    
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [15]:
# 상위 10개국 bar 차트

# 총 확진자
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [16]:
# 총 사망자
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [17]:
# 총 완치자 

px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [18]:
# 전 세계 사망률 확인

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=1, max_zoom = 6, min_zoom = 1)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(deaths_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(deaths_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map